In [ ]:
from db_import import *
from datetime import datetime, timedelta
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 30)
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Data Import
end_date = datetime.now().strftime('%Y-%m-%d')
query_obj = Queries('2022-01-01', end_date)
db_obj = DBImport(db_type='cscart')

analytics = db_obj.data_import(query_obj.analytics_query)
brand = db_obj.data_import(query_obj.brand_query)
inventory = db_obj.data_import(query_obj.inventory_query)
category = db_obj.data_import(query_obj.category_query)
mainexposure = db_obj.data_import(query_obj.mainexposure_query)

In [ ]:
# basic preprocessing
analytics['purchased_ymd'] = pd.to_datetime(analytics.purchased_at).dt.normalize() # 시간 제외한 날짜만
brand['brand'] = brand['brand'].apply(lambda x: x.strip())

In [ ]:
# product-barcode matching table
prod_bar_match = analytics[['product_id', 'barcode']].drop_duplicates()

# barcode-product_name matching table (추후 바코드 없는 상품들은 바코드 기준 조인이 안되기 때문에 상품명, 옵션명 테이블 따로 생성)
bar_prod_name = analytics[['product_id', 'product_name_kor']].drop_duplicates()
bar_prod_name = bar_prod_name.groupby('product_id')['product_name_kor'].sum().reset_index() # 한 상품명 내에서 여러 옵션이 담겨있기도 함. 모든 옵션을 보여주기 위해 합치기

# barcode-variant_name matching table (추후 바코드 없는 상품들은 바코드 기준 조인이 안되기 때문에 상품명, 옵션명 테이블 따로 생성)
bar_var_name = analytics[['barcode', 'variant_1_name_kor']].drop_duplicates()
bar_var_name = bar_var_name.groupby('barcode')['variant_1_name_kor'].sum().reset_index() # 한 바코드 내에서 여러 옵션이 담겨있기도 함. 모든 옵션을 보여주기 위해 합치기

# product_id-vendor matching table
prod_vendor = analytics[['product_id', 'company_name']].drop_duplicates()

# 판매 관련 확인

- 판매 급상승의 경우, 과거 판매가 된 날짜만을 바탕으로 진행. 즉, 재고가 있지만 판매가 일어나지 않아 **판매량을 0으로 집계하는 과정은 생략 (∵ ① 지나친 저회전 상품에 대해 급상승을 보고싶은 것도 아니고, ② 재고 정보가 부정확할 수도 있음. 무엇보다도, ③ 0을 복구해줌으로서 분모에 위치시킬 경우 왜곡이 발생할 수 있기 때문)**
- 현재 날짜를 기준으로 최근 일주일을 **타겟기간**, 일주일전으로부터 14일 전까지를 **비교기간**으로 설정 → 각 기간 내에서도 OOS로 인해 판매가 일어나지 않았을 경우, 단순 합으로 비교가 어렵기 때문에 주단위 판매량 합이 아니라, **각 기간의 일평균**으로 연산
- 위와같이 설정할 경우, 비교기간동안 판매가 0건인 상품의 경우 inf로 연산되기 때문에 **비교기간동안 판매가 없는 상품**의 경우 별도 처리 필요 → **1로 대체** (∵ 최근 입고 등의 이슈로 최근 판매량이 급상승 한 상품은 주로 추적을 원하기 때문에, 판매수량이 그대로 입력되도록 분모를 1로 설정. 단, 판매량이 어느정도 되어야 재입고 급상승의 의미가 있기 때문에, **타겟기간 최근 판매량에 threshold 설정**)
- ~~상대적으로 중요도가 떨어지는, **판매량이 낮은 상품들**도 비율상으로 커져보일 수 있음. **비교기간 평균판매량에 threshold 설정**~~ → 제외. 바로 위에서 최근 재입고 상품의 경우 분모가 1이되어야 하는데, 비교기간 평균판매량에 threshold가 1보다 크게 걸리면 해당 상품들은 제외되기 때문
- 카테고리의 경우, 기존의 category_M, category_S 두 카테고리를 활용하는 것이 아니라, 상품:카테고리의 1:N 매칭이 이루어지도록 카테고리 테이블을 활용하여 한 상품이 여러 카테고리에 중복 집계가 가능하도록 연산
- 브랜드, 카테고리별 결과의 경우 상위 10개의 결과에 해당하는 상품별 상세정보 테이블 추가로 생성

In [ ]:
# target and compare dates
target_end = datetime.now().strftime('%Y-%m-%d')
target_start = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
compare_end = (datetime.now() - timedelta(days=8)).strftime('%Y-%m-%d')
compare_start = (datetime.now() - timedelta(days=15)).strftime('%Y-%m-%d')
target_compare_days = [compare_start, compare_end, target_start, target_end] # 원인확인 class 생성시 필요

In [ ]:
### aggregation

## 1. 상품별 결과
target_qty_threshold = 5 # 타겟기간 평균판매량의 threshold. 급상승에서 타겟기간 최소판매기준
compare_qty_threshold = 5 # 비교기간 평균판매량의 threshold. 급하락에서 과거기간 최소판매기준

# barcode 있는 상품들
daily_sales = analytics.groupby(['barcode', 'purchased_ymd'])['product_qty'].sum().reset_index()
daily_sales = daily_sales[~(daily_sales['barcode']=='')]
target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('barcode').agg({'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'product_qty':'target_qty', 'purchased_ymd':'target_days'})
compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('barcode').agg({'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})
prod_sales = pd.merge(target_sales, compare_sales, on='barcode', how='outer')
prod_sales = prod_sales.reset_index()
prod_sales = pd.merge(prod_sales, prod_bar_match, on='barcode', how='left')

# barcode 없는 국내 일부 상품들
non_barcode = analytics[(analytics['barcode']=='')|(analytics['barcode'].isnull())]
target_sales_non = non_barcode[(non_barcode['purchased_ymd']>target_start)&(non_barcode['purchased_ymd']<=target_end)]
target_sales_non = target_sales_non.groupby('product_id').agg({'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
target_sales_non = target_sales_non.rename(columns={'product_qty':'target_qty', 'purchased_ymd':'target_days'})
compare_sales_non = non_barcode[(non_barcode['purchased_ymd']>compare_start)&(non_barcode['purchased_ymd']<=compare_end)]
compare_sales_non = compare_sales_non.groupby('product_id').agg({'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
compare_sales_non = compare_sales_non.rename(columns={'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})
prod_sales_non = pd.merge(target_sales_non, compare_sales_non, on='product_id', how='outer')
prod_sales_non = prod_sales_non.reset_index()

prod_sales = pd.concat([prod_sales, prod_sales_non])

# growth_rate 성장률 계산을 위해서는 판매량과 판매일자를 1로 채워줘야 하는데, 실제 값은 0으로 해야하기 때문에 저장해두고 추후 복구
target_days = prod_sales['target_days'].fillna(0)
compare_qty = prod_sales['compare_qty'].fillna(0)
compare_days = prod_sales['compare_days'].fillna(0)
prod_sales['target_qty'] = prod_sales['target_qty'].fillna(0) # target_qty는 0 그대로여도 상관없음. 항상 최우선 분자에 있기 때문. 나머지는 1로 채우기
prod_sales['target_days'] = prod_sales['target_days'].fillna(1) 
prod_sales['compare_qty'] = prod_sales['compare_qty'].fillna(1)
prod_sales['compare_days'] = prod_sales['compare_days'].fillna(1)
prod_sales['growth_rate_daily'] = (prod_sales['target_qty']/prod_sales['target_days']) / (prod_sales['compare_qty']/prod_sales['compare_days'])
prod_sales['target_days'] = target_days
prod_sales['compare_qty'] = compare_qty
prod_sales['compare_days'] = compare_days

# 후처리
prod_sales['compare_qty'] = prod_sales['compare_qty'].astype('int')
prod_sales['compare_days'] = prod_sales['compare_days'].astype('int')
prod_sales['growth_rate_daily'] = np.round(prod_sales['growth_rate_daily'], 2)
prod_sales = pd.merge(prod_sales, brand, on='product_id', how='left')
prod_sales = pd.merge(prod_sales, bar_prod_name, on='product_id', how='left') # barcode 없는 상품들로 인해 상품명과 옵션명 별도로 조인
prod_sales = pd.merge(prod_sales, bar_var_name, on='barcode', how='left')

# 최종결과
prod_top20 = prod_sales[prod_sales['target_qty']>=target_qty_threshold].sort_values(by=['growth_rate_daily', 'target_qty'], ascending=[False, False]).head(20) # 성장률 우선정렬, 다음은 qty 기준
prod_bot20 = prod_sales[prod_sales['compare_qty']>=compare_qty_threshold].sort_values(by=['growth_rate_daily', 'compare_qty', 'compare_days'], ascending=[True, False, True]).head(20)

In [ ]:
class CauseTracker():
    def __init__(self, mainexposure, inventory, target_compare_days):
        self.mainexposure = mainexposure
        self.inventory = inventory
        self.target_compare_days = target_compare_days
        self.col_name_dict =  {'barcode': '바코드', # 추후 컬럼명 한국어로 변경 시 공통 활용되는 dictionary
                               'target_qty': '최근일주일판매량',
                               'target_days': '최근일주일판매일수',
                               'compare_qty': '과거일주일판매량',
                               'compare_days': '과거일주일판매일수',
                               'growth_rate_daily': '판매량상승률',
                               'product_id': '상품번호',
                               'brand': '브랜드',
                               'product_name_kor': '상품명',
                               'variant_1_name_kor': '옵션명',
                               'target_category': '최근일주일노출영역',
                               'compare_category': '과거일주일노출영역',
                               'exposure_comment': '원인_메인노출여부',
                               'target_amount': '현재고',
                               'compare_amount': '과거일주일재고(MAX)',
                               'amt_gap': '재고변동',
                               'inventory_comment': '원인_재고여부',
                               'category': '카테고리',
                               'company_name': '벤더사'
                              }

    def comment_exposure(self, arr):
        if (arr['target_category']=='-')&(arr['compare_category']=='-'):
            return '항상노출안함'
        elif (arr['target_category']!='-')&(arr['compare_category']=='-'):
            return '메인노출'
        elif (arr['target_category']=='-')&(arr['compare_category']!='-'):
            return '노출제외'
        elif (arr['target_category']!='-')&(arr['compare_category']!='-'):
            return '항상노출'

    def comment_inventory(self, arr):
        oos_amount_threshold = 0 # 재고비교 시 사실상 결품이라고 인정할 정도의 재고 수량이 얼마 이하로 할 것인지
        if (arr['compare_amount']<=oos_amount_threshold)&(arr['target_amount']>arr['compare_amount']):
            return '재입고'
        elif (arr['compare_amount']>oos_amount_threshold)&(arr['target_amount']>arr['compare_amount']):
            return '재고보충'
        elif (arr['target_amount']<=oos_amount_threshold)&(arr['target_amount']<arr['compare_amount']):
            return '품절'
        elif (arr['target_amount']>oos_amount_threshold)&(arr['target_amount']<arr['compare_amount']):
            return '재고감소'
        else:
            return '-'

    def make_exposure_cause(self, result_df):
        # main exposure table
        target_expose = self.mainexposure[(self.mainexposure['date']>self.target_compare_days[2])&(self.mainexposure['date']<=self.target_compare_days[3])]
        target_expose = target_expose.sort_values(by='date', ascending=False).drop_duplicates(subset='product_id')
        target_expose = target_expose.rename(columns={'main_rec_category': 'target_category'})
        compare_expose = self.mainexposure[(self.mainexposure['date']>self.target_compare_days[0])&(self.mainexposure['date']<=self.target_compare_days[1])]
        compare_expose = compare_expose.sort_values(by='date', ascending=False).drop_duplicates(subset='product_id')
        compare_expose = compare_expose.rename(columns={'main_rec_category': 'compare_category'})
        prod_expose = pd.merge(target_expose[['product_id', 'target_category']], compare_expose[['product_id', 'compare_category']], on='product_id', how='outer')

        result_df = pd.merge(result_df, prod_expose, on='product_id', how='left')
        result_df[['target_category', 'compare_category']] = result_df[['target_category', 'compare_category']].fillna('-')
        result_df['exposure_comment']=result_df[['target_category', 'compare_category']].apply(lambda x: self.comment_exposure(x), axis=1)
        return result_df

    def make_inventory_cause(self, result_df):
        # main exposure table
        target_inventory = self.inventory[(self.inventory['sys_time']>self.target_compare_days[2])&(self.inventory['sys_time']<=self.target_compare_days[3])]
        compare_inventory = self.inventory[(self.inventory['sys_time']>self.target_compare_days[0])&(self.inventory['sys_time']<=self.target_compare_days[1])]
        # 재고는 일단위 갱신정보이기 때문에, 합산이 불가하고 특정 날짜 기준으로 보아야 하는데, 현재고가 중요하니 타겟기간은 현재고, 비교기간은 최대한 보수적인 기준으로 max 재고였던날 기준으로 확인
        target_inventory = target_inventory[target_inventory['sys_time']==target_inventory['sys_time'].max()]
        target_inventory = target_inventory.groupby('barcode')['amount'].max().reset_index()
        target_inventory = target_inventory.rename(columns={'amount': 'target_amount'})
        compare_inventory = compare_inventory.groupby('barcode')['amount'].max().reset_index()
        compare_inventory = compare_inventory.rename(columns={'amount': 'compare_amount'})
        bar_inventory = pd.merge(target_inventory, compare_inventory, on='barcode', how='outer')
        bar_inventory = bar_inventory.fillna(0)
        bar_inventory['amt_gap'] = bar_inventory['target_amount'] - bar_inventory['compare_amount']

        result_df = pd.merge(result_df, bar_inventory, on='barcode', how='left')
        result_df['inventory_comment']=result_df[['target_amount', 'compare_amount']].apply(lambda x: self.comment_inventory(x), axis=1)
        return result_df

    def prod_cause(self, result_df):
        result_df = result_df
        result_df = self.make_exposure_cause(result_df)
        result_df = self.make_inventory_cause(result_df)
        result_df = result_df[[
            'product_name_kor', 'variant_1_name_kor', 'brand', 'growth_rate_daily', 'target_qty', 'compare_qty', 'target_amount', 'compare_amount',
            'inventory_comment', 'exposure_comment', 'amt_gap', 'target_category', 'compare_category', 'target_days', 'compare_days', 'product_id', 'barcode'
        ]] # 순서 변경
        result_df = result_df.rename(columns=self.col_name_dict)
        result_df.index = result_df.index + 1
        result_df = result_df.fillna('-')
        return result_df

    def brand_cause(self, result_df, prod_df): # @@@ 상품별 급상승/하락 정보를 담고있는 prod_df는 필수 정보. 따라서, 상품별 급상승하락 연산 로직 또한 main.py가 아니라 이 class안에 넣는게 좋을 듯
        brand_detail = []
        for brand_target in result_df['brand'].tolist():
            brand_prod = prod_df[prod_df['brand']==brand_target]
            brand_prod = brand_prod.sort_values(by='growth_rate_daily', ascending=False)
            brand_detail.append(brand_prod)
        brand_detail = pd.concat(brand_detail, ignore_index=True)
        brand_detail = self.make_exposure_cause(brand_detail)
        brand_detail = self.make_inventory_cause(brand_detail)
        brand_detail = brand_detail[[
            'brand', 'product_name_kor', 'variant_1_name_kor', 'growth_rate_daily', 'target_qty', 'compare_qty', 'target_amount', 'compare_amount',
            'inventory_comment', 'exposure_comment', 'amt_gap', 'target_category', 'compare_category', 'target_days', 'compare_days', 'product_id', 'barcode'
        ]] # 순서 변경
        brand_detail = brand_detail.rename(columns=self.col_name_dict)
        brand_detail.index = brand_detail.index + 1
        brand_detail = brand_detail.fillna('-')
        return brand_detail

    def cate_cause(self, result_df, prod_df, category_df): # @@@ 상품별 급상승/하락 정보를 담고있는 prod_df는 필수 정보. 따라서, 상품별 급상승하락 연산 로직 또한 main.py가 아니라 이 class안에 넣는게 좋을 듯
        prod_df = pd.merge(prod_df, category, on='product_id', how='left') # 카테고리 정보를 미리 prod_sales에 merge할 수 없는 이유가, 카테고리는 1:N 매칭이기 때문. 따라서 미리 merge하면 top을 뽑을 때 문제 발생. cause 파악 단계에서만 merge할 것
        cate_detail = []
        for cate_target in result_df['category'].tolist():
            cate_prod = prod_df[prod_df['category']==cate_target]
            cate_prod = cate_prod.sort_values(by='growth_rate_daily', ascending=False)
            cate_detail.append(cate_prod)
        cate_detail = pd.concat(cate_detail, ignore_index=True)
        cate_detail = self.make_exposure_cause(cate_detail)
        cate_detail = self.make_inventory_cause(cate_detail)
        cate_detail = cate_detail[[
            'category', 'product_name_kor', 'variant_1_name_kor', 'brand', 'growth_rate_daily', 'target_qty', 'compare_qty', 'target_amount', 'compare_amount',
            'inventory_comment', 'exposure_comment', 'amt_gap', 'target_category', 'compare_category', 'target_days', 'compare_days', 'product_id', 'barcode'
        ]] # 순서 변경
        cate_detail = cate_detail.rename(columns=self.col_name_dict)
        cate_detail.index = cate_detail.index + 1
        cate_detail = cate_detail.fillna('-')
        return cate_detail

    def vendor_cause(self, result_df, prod_df, vendor_df): # @@@ 상품별 급상승/하락 정보를 담고있는 prod_df는 필수 정보. 따라서, 상품별 급상승하락 연산 로직 또한 main.py가 아니라 이 class안에 넣는게 좋을 듯
        prod_df = pd.merge(prod_df, vendor_df, on='product_id', how='left')
        vendor_detail = []
        for vendor_target in result_df['company_name'].tolist():
            vendor_prod = prod_df[prod_df['company_name']==vendor_target]
            vendor_prod = vendor_prod.sort_values(by='growth_rate_daily', ascending=False)
            vendor_detail.append(vendor_prod)
        vendor_detail = pd.concat(vendor_detail, ignore_index=True)
        vendor_detail = self.make_exposure_cause(vendor_detail)
        vendor_detail = self.make_inventory_cause(vendor_detail)
        vendor_detail = vendor_detail[[
            'company_name', 'product_name_kor', 'variant_1_name_kor', 'brand', 'growth_rate_daily', 'target_qty', 'compare_qty', 'target_amount', 'compare_amount',
            'inventory_comment', 'exposure_comment', 'amt_gap', 'target_category', 'compare_category', 'target_days', 'compare_days', 'product_id', 'barcode'
        ]] # 순서 변경
        vendor_detail = vendor_detail.rename(columns=self.col_name_dict)
        vendor_detail.index = vendor_detail.index + 1
        vendor_detail = vendor_detail.fillna('-')
        return vendor_detail

In [ ]:
tracker = CauseTracker(mainexposure, inventory, target_compare_days)
tracker.prod_cause(prod_bot20)

## 2. 브랜드별 결과

In [ ]:
## 2. 브랜드별 결과
target_qty_threshold = 5 # 비교기간 판매가 없는 상품의 타겟기간 평균판매량의 threshold
compare_qty_threshold = 5 # 비교기간 평균판매량의 threshold

daily_sales = analytics.groupby(['product_id', 'purchased_ymd'])['product_qty'].sum().reset_index()
daily_sales = pd.merge(daily_sales, brand, on='product_id', how='left')
daily_sales = daily_sales[daily_sales['brand'].notnull()] # 브랜드별 결과이기 때문에, 브랜드 정보 없는 상품은 제외 (단, 브랜드 쿼리 및 테이블 소스 다시 한번 DBMS상에서 확인해볼 것)

target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('brand').agg({'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'product_qty':'target_qty', 'purchased_ymd':'target_days'})
compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('brand').agg({'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})
brand_sales = pd.merge(target_sales, compare_sales, on='brand', how='outer')
brand_sales = brand_sales.reset_index()

# growth_rate 성장률 계산을 위해서는 판매량과 판매일자를 1로 채워줘야 하는데, 실제 값은 0으로 해야하기 때문에 저장해두고 추후 복구
target_days = brand_sales['target_days'].fillna(0)
compare_qty = brand_sales['compare_qty'].fillna(0)
compare_days = brand_sales['compare_days'].fillna(0)
brand_sales['target_qty'] = brand_sales['target_qty'].fillna(0) # target_qty는 0 그대로여도 상관없음. 항상 최우선 분자에 있기 때문. 나머지는 1로 채우기
brand_sales['target_days'] = brand_sales['target_days'].fillna(1)
brand_sales['compare_qty'] = brand_sales['compare_qty'].fillna(1)
brand_sales['compare_days'] = brand_sales['compare_days'].fillna(1)
brand_sales['growth_rate_daily'] = (brand_sales['target_qty']/brand_sales['target_days']) / (brand_sales['compare_qty']/brand_sales['compare_days'])
brand_sales['target_days'] = target_days
brand_sales['compare_qty'] = compare_qty
brand_sales['compare_days'] = compare_days

# 후처리
brand_sales['compare_qty'] = brand_sales['compare_qty'].astype('int')
brand_sales['compare_days'] = brand_sales['compare_days'].astype('int')
brand_sales['growth_rate_daily'] = np.round(brand_sales['growth_rate_daily'], 2)

# 최종결과
brand_top20 = brand_sales[brand_sales['target_qty']>=target_qty_threshold].sort_values(by=['growth_rate_daily', 'target_qty'], ascending=[False, False]).head(20) # 성장률 우선정렬, 다음은 qty 기준
brand_bot20 = brand_sales[brand_sales['compare_qty']>=compare_qty_threshold].sort_values(by=['growth_rate_daily', 'compare_qty', 'compare_days'], ascending=[True, False, True]).head(20)

In [ ]:
brand_top20

In [ ]:
tracker.brand_cause(brand_top20, prod_sales)

## 3. 카테고리별 결과

In [ ]:
## 3. 카테고리별 결과
target_qty_threshold = 5 # 비교기간 판매가 없는 상품의 타겟기간 평균판매량의 threshold
compare_qty_threshold = 5 # 비교기간 평균판매량의 threshold

daily_sales = analytics.groupby(['product_id', 'purchased_ymd'])['product_qty'].sum().reset_index()
daily_sales = pd.merge(daily_sales, category, on='product_id', how='left')
daily_sales = daily_sales[daily_sales['category'].notnull()] # 카테고리별 결과이기 때문에, 카테고리 정보 없는 상품은 제외

target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('category').agg({'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'product_qty':'target_qty', 'purchased_ymd':'target_days'})
compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('category').agg({'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})
cate_sales = pd.merge(target_sales, compare_sales, on='category', how='outer')
cate_sales = cate_sales.reset_index()

# growth_rate 성장률 계산을 위해서는 판매량과 판매일자를 1로 채워줘야 하는데, 실제 값은 0으로 해야하기 때문에 저장해두고 추후 복구
target_days = cate_sales['target_days'].fillna(0)
compare_qty = cate_sales['compare_qty'].fillna(0)
compare_days = cate_sales['compare_days'].fillna(0)
cate_sales['target_qty'] = cate_sales['target_qty'].fillna(0) # target_qty는 0 그대로여도 상관없음. 항상 최우선 분자에 있기 때문. 나머지는 1로 채우기
cate_sales['target_days'] = cate_sales['target_days'].fillna(1)
cate_sales['compare_qty'] = cate_sales['compare_qty'].fillna(1)
cate_sales['compare_days'] = cate_sales['compare_days'].fillna(1)
cate_sales['growth_rate_daily'] = (cate_sales['target_qty']/cate_sales['target_days']) / (cate_sales['compare_qty']/cate_sales['compare_days'])
cate_sales['target_days'] = target_days
cate_sales['compare_qty'] = compare_qty
cate_sales['compare_days'] = compare_days

# 후처리
cate_sales['compare_qty'] = cate_sales['compare_qty'].astype('int')
cate_sales['compare_days'] = cate_sales['compare_days'].astype('int')
cate_sales['growth_rate_daily'] = np.round(cate_sales['growth_rate_daily'], 2)

# 최종결과
cate_top20 = cate_sales[cate_sales['target_qty']>=target_qty_threshold].sort_values(by=['growth_rate_daily', 'target_qty'], ascending=[False, False]).head(20) # 성장률 우선정렬, 다음은 qty 기준
cate_bot20 = cate_sales[cate_sales['compare_qty']>=compare_qty_threshold].sort_values(by=['growth_rate_daily', 'compare_qty', 'compare_days'], ascending=[True, False, True]).head(20)

In [ ]:
cate_top20

In [ ]:
tracker.cate_cause(cate_top20, prod_sales, category)

## 4. 벤더별 결과

In [ ]:
## 4. 벤더별 결과
target_qty_threshold = 5 # 비교기간 판매가 없는 상품의 타겟기간 평균판매량의 threshold
compare_qty_threshold = 5 # 비교기간 평균판매량의 threshold

daily_sales = analytics[analytics['company_name'].notnull()] # 벤더별 결과이기 때문에, 벤더 정보 없는 상품은 제외

target_sales = daily_sales[(daily_sales['purchased_ymd']>target_start)&(daily_sales['purchased_ymd']<=target_end)]
target_sales = target_sales.groupby('company_name').agg({'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
target_sales = target_sales.rename(columns={'product_qty':'target_qty', 'purchased_ymd':'target_days'})
compare_sales = daily_sales[(daily_sales['purchased_ymd']>compare_start)&(daily_sales['purchased_ymd']<=compare_end)]
compare_sales = compare_sales.groupby('company_name').agg({'product_qty':'sum', 'purchased_ymd': pd.Series.nunique})
compare_sales = compare_sales.rename(columns={'product_qty':'compare_qty', 'purchased_ymd':'compare_days'})
vendor_sales = pd.merge(target_sales, compare_sales, on='company_name', how='outer')
vendor_sales = vendor_sales.reset_index()

# growth_rate 성장률 계산을 위해서는 판매량과 판매일자를 1로 채워줘야 하는데, 실제 값은 0으로 해야하기 때문에 저장해두고 추후 복구
target_days = vendor_sales['target_days'].fillna(0)
compare_qty = vendor_sales['compare_qty'].fillna(0)
compare_days = vendor_sales['compare_days'].fillna(0)
vendor_sales['target_qty'] = vendor_sales['target_qty'].fillna(0) # target_qty는 0 그대로여도 상관없음. 항상 최우선 분자에 있기 때문. 나머지는 1로 채우기
vendor_sales['target_days'] = vendor_sales['target_days'].fillna(1)
vendor_sales['compare_qty'] = vendor_sales['compare_qty'].fillna(1)
vendor_sales['compare_days'] = vendor_sales['compare_days'].fillna(1)
vendor_sales['growth_rate_daily'] = (vendor_sales['target_qty']/vendor_sales['target_days']) / (vendor_sales['compare_qty']/vendor_sales['compare_days'])
vendor_sales['target_days'] = target_days
vendor_sales['compare_qty'] = compare_qty
vendor_sales['compare_days'] = compare_days

# 후처리
vendor_sales['compare_qty'] = vendor_sales['compare_qty'].astype('int')
vendor_sales['compare_days'] = vendor_sales['compare_days'].astype('int')
vendor_sales['growth_rate_daily'] = np.round(vendor_sales['growth_rate_daily'], 2)

# 최종결과
vendor_top20 = vendor_sales[vendor_sales['target_qty']>=target_qty_threshold].sort_values(by=['growth_rate_daily', 'target_qty'], ascending=[False, False]).head(20) # 성장률 우선정렬, 다음은 qty 기준
vendor_bot20 = vendor_sales[vendor_sales['compare_qty']>=compare_qty_threshold].sort_values(by=['growth_rate_daily', 'compare_qty', 'compare_days'], ascending=[True, False, True]).head(20)

In [ ]:
vendor_top20

In [ ]:
tracker.vendor_cause(vendor_top20, prod_sales, prod_vendor)

In [ ]:
# file save
today = datetime.today().strftime('%Y%m%d')
path = '{}_급상승하락.xlsx'.format(today)
writer = pd.ExcelWriter(path, engine='xlsxwriter')

tracker.cause_comment(prod_top20).to_excel(writer, sheet_name='급상승')
tracker.cause_comment(prod_bot20).to_excel(writer, sheet_name='급하락')

writer.save()
writer.close()